# Optimization Framework 
This prototype aims to identify the optimal parameter for an interest rate model in a LlammaLend market. In this notebook we do not deal with the question of an optimal Utilisation. We use Bertucci et al. (2024) as foundation of this work. The authors formualated IRM parameterisation as stochastic optimal control problem. In practical terms this means, Bertucci et al. used a methodology that can be summarised as: 
1. **Desiging mathematical model** encapsulating Borrower and supplier behaviour which is reduced to the utilisation ratio.
2. **Formulation of an Optimisation Problem** encabsulating the our objective. While utilimately many goals are viable we will focus on the closeness to target utlisation. 
3. **Attempt an Analytical Solution using HJB**. Use the dynamic programming principle to derive the HJB equation, which characterizes the value function as a partial differential equation (PDE). 
4. **Solve the HJB equation numerically** Since the HJB equation is often nonlinear and high-dimensional, exact solutions are rare. While mutliple methods exist to arrive at an answer we will be simulate stochastic processes to estimate the value function.


The goal of this notebook is to exemplify and validate the application to a the:
- crv <> crvUSD Market in LlammaLend
- sUSD <> crvUSD Market in LlammaLend

While in the last notebook we focussed on quick implementation in this notebook we will try to rigioursly outline the theoretical underpinnings, outline assumption and show needed validation when applicable. 

> Note: It is worth noting that the original model was developed aToken based models - yet in this case the distiction should not matter any further.

##  Desiging the mathematical model

### Background 
Developing robust mathematical models is hard - henceforth in my option it is advisable to consult the academic literature. Bertucci et al.,(2024) outlines the following model which while not perfect but is reasonable. 

The author begin with the assumption that borrower $B_t$ and suppliers $S_t$ do not react to the instanteous rate at given block but to dynamic averages over time. This is their justification to model $B_t$ and $S_t$ as "solutions to stochastic differential equations, where the dynamics are fundamentally driven by interest rates.".

They introduce two mathematical model, one 2-dimensional model, explicitly modelling $B_t$ and $S_t$ and a 1-dimensional model purely on the utilisation rate. We, as well as the author, will focus on the one-dimensional model, as the utlisation rate $U_t$ is a function of $B_t$ and $S_t$ and for the practical reason that DeFi platforms, as well as the current monetrary polices utiised by [LlammaLend](https://docs.curve.fi/lending/contracts/mp-overview/) primarily rely on the utilization ratio to set the rate. 

### The Mathematical Model
The utilisation $U_t$ is our state variable, that is it describes the current state of our system. It is given by: 
$$dU_t = f_U(t, U_t, r_t) \, dt + \sigma_U(t, U_t, r_t) \, dW_t$$
, where: 
- $f_U(t, U_t, r_t) \, dt$ is the drift term,representing the deterministic part of the model. This term captures how $U_t$ evolves predictably over time due to factors like interest rates and system parameters.It captures how $U_t$ 
evolves predictably over time due to factors like interest rates and system parameters. It is a function of:
    - $t$: The current time.
    - $U_t$: The current utilization ratio.
    - $r_t$: The current interest rate.
- $\sigma_U(t, U_t, r_t) \, dW_t$: the diffusion term, representing the random fluctuations in $U_t$. it incorporates: 
    - $\sigma_U(t, U_t, r_t)$: The volatility of $U_t$ which depends on time, the current utilization ratio, and the interest rate.
    - $dW_t$ A standard Brownian motion term, which introduces randomness into the model, capturing unpredictable changes in $U_t$

#### Control Variables 
The control variable, that is the decision variable we need to optimise in the objective function is the $r_t$, which is a function of utitisation or our interest rate policy, $r_t = r(t, U_t)$. 

#### Objective function
Given that we assume we can identify $U_{optimal}$ as target, we choose as ojective to minimizing the deviation from a target utilization ratio $U_{target}$ to maximizing $U_t$ at each time $t$ (a.k.a. we choose the Target Utilization Ratio Model Eq. 4). The objective function is given by a stationary variant model, this means we try to optimise over an indefinite time horizon with a discount rate $\lambda > 0$, unlike a finite horizon model: 

$$
\inf_{r \in \mathcal{A}^\infty_U} \mathbb{E} \left[ \int_{0}^{\tau_U} e^{-\lambda t} \left( a(U_t - U_{\text{target}})^2 + \frac{c}{2} (r_t - \bar{r})^2 \right) dt \right]
$$

where:
- $\mathcal{A}^\infty_U = \{ (r_t)_{t \geq 0} : \text{nonnegative, adapted, and} \ \mathbb{E}[\int_{0}^{\tau_U} e^{-\lambda t} r_t^2 dt] < \infty, \ \forall t \leq \tau_U, \ U_t < 1 \ \text{a.s.} \}$

> **Explanation for dummies**
> If you didn't have clue what this mean or why it looks this way, me neither when I first read it. Effectively, it follows a typical Stochastic Optimal Control Cost Function.

>What does a typical typical Stochastic Optimal Control Cost Function look like: 
>1. **Objective:**
>   - The cost function aims to minimize an expected cumulative cost over a time horizon, often with a stopping time or constraint ($\tau_U$ here).
>2. **Discount Factor ($e^{-\lambda t}$):**
>   - Common in infinite-horizon problems to prioritize immediate costs over future costs and ensure the total cost remains finite. The discount rate $\lambda > 0$ reflects the time value of cost.
>3. **State Variable Penalty ($a(U_t - U_{\text{target}})^2$):**
>   - Quadratic penalties on the deviation of the state $U_t$ from a desired target $U_{\text{target}}$ are standard, as they ensure smooth optimization and penalize larger deviations more heavily.
>4. **Control Variable Penalty ($\frac{c}{2} (r_t - \bar{r})^2$):**
>   - A quadratic penalty on the control $r_t$ relative to a reference $\bar{r}$ is typical to prevent excessive or unrealistic control effort. This term balances the optimization and prevents over-aggressive or erratic behavior in the control variable.
>5. **Expectation ($\mathbb{E}$):**
>   - The cost function is stochastic, meaning randomness in $U_t$ and $r_t$ (e.g., due to a Brownian motion or other stochastic drivers) necessitates taking the expectation to account for uncertainty in the outcomes.
>6. **Admissible Controls ($r \in \mathcal{A}^\infty_U$):**
>   - Constraints on the control $r_t$ ensure feasibility (e.g., $r_t$ must be nonnegative and adapted) and prevent solutions that are unrealistic or overly costly in practice.

>Let break it down a Equation itself to understand what it does: 
> 1. **Objective**:
>   - Minimize the total cost combining:
>     - $a(U_t - U_{\text{target}})^2$: Penalizes deviations of the utilization $U_t$ from the target $U_{\text{target}}$.
>     - $\frac{c}{2}(r_t - \bar{r})^2$: Penalizes deviations of the control $r_t$ from the reference rate $\bar{r}$.
> 2. **Stationary Variant**:
>   - Includes a discount factor $e^{-\lambda t}$, prioritizing immediate costs over long-term costs.
> 3. **Constraints**:
>   - $ r_t$: Must be nonnegative and adapted (depends only on available information).
>   - $U_t < 1$: Utilization must remain below 1 (e.g., avoiding overutilization).

**Remark**
You may wonder, why is the cost function cost $(r_t - \bar{r})^2$. Bertucci et al., acknowledge their arbitrary choice of a quadratic cost functional $(r_t - \bar{r})^2_+$, emphasizing that different powers $q$ would significantly alter the behavior of the optimal control, with $q = 2$ chosen for clarity and its critical role in balancing practicality and theoretical implications.



### Mathematical Analysis
To-do:
- [ ] Work through the math 

For the sake of briefty please refer pages 8 to 15 of Bertucci et al. for the mathematical analysis of the one-dimensional model (i.e. based on a utilisation). 

Tl;dr - The section maps out boundary conditions of the mathematical model, shows the critera that allows to apply HJB and the application of HJB. The model - as it is common with this methodology does not find a exact solutions, meaning we will apply numerical methods to estimate solutions.

<!-- Bertucci et al. (2024) introduces the model based on Utilisation for which they conduct a mathematical analysis. It is reformulated as a more tracebale problem by reformulating it as $X=1−U$.

#### What is the goal of the mathematical analysis? 
We are minimizing a cost function over time for a controlled system. The cost has two main components:

1. State cost $f(X_t)$: Penalises the system for being in undesriable state $X_t$ derived from $U_t$
2. Control cost $(r_t - \bar{r})^2$: Penalises deviation from the control variable $r_t$ (e.g. interest rate) from a target value $\bar{r}$

The goal is to find an optimal control strategy $r_t$ that minimizes the expected discounted cost over time. -->




### Numerical Solution of HJB 

We first arrive at numerical solution of the HJB solution. Why? This answer the question under perfect knowledge how well the interest rate could be controlled, in practice this is not viable as all inputs cannot be perfectly know.

The usefulness of this approach is that we can benchmark the "gold standard" to PID implementation (i.e. linear piece wise functions, semi-log monetary policy). 

In [ ]:
### Numerical Analysis of PID Control 


In [27]:
import numpy as np

class LinearPiecewiseController:
    def __init__(self, u_target, phi_params):
        self.u_target = u_target
        self.phi_params = phi_params  # Parameters for phi function

    def adjust_rate(self, u):
        delta_u = u - self.u_target
        if delta_u <= -0.05:
            return self.phi_params['low']  # Increase rate by 0.9
        elif delta_u >= 0.05:
            return self.phi_params['high']  # Decrease rate by 1.1
        else:
            return self.phi_params['neutral']  # No change

class AdaptiveIrmController:
    def __init__(self, kp, kd, u_target):
        self.kp = kp
        self.kd = kd
        self.u_target = u_target

    def err(self, u):
        if u >= self.u_target:
            return (u - self.u_target) / (1 - self.u_target)
        else:
            return (u - self.u_target) / self.u_target

    def xi(self, u, err):
        if u >= self.u_target:
            return (self.kd - 1) * err + 1
        else:
            return ((1 - 1 / self.kd) * err) + 1

    def adjust_rate(self, u, du_dt):
        err = self.err(u)
        xi_prime = self.xi(u, err)
        return self.kp * err + xi_prime * du_dt

class SemiLogController:
    def __init__(self, base_rate, multiplier, u_target):
        self.base_rate = base_rate  # Base interest rate
        self.multiplier = multiplier  # Multiplier for exponential growth
        self.u_target = u_target  # Target utilization ratio

    def adjust_rate(self, u):
        if u <= self.u_target:
            return self.base_rate * np.exp(self.multiplier * (u - self.u_target))
        else:
            return self.base_rate * np.exp(self.multiplier * (self.u_target - u))


In [26]:
class Simulation:
    def __init__(self, controller, params, dt=0.01):
        self.controller = controller
        self.params = params
        self.dt = dt
        self.u = params['u_init']
        self.r = params['r_init']
        self.u_history = []
        self.r_history = []

    def step(self):
        # Dynamics of utilization ratio
        drift = self.params['alpha0'] + self.params['alpha1'] * self.r
        du_dt = drift * self.dt
        self.u += du_dt

        # Adjust interest rate
        if isinstance(self.controller, LinearPiecewiseController):
            dr_dt = self.controller.adjust_rate(self.u)
        elif isinstance(self.controller, AdaptiveIrmController):
            dr_dt = self.controller.adjust_rate(self.u, du_dt)
        elif isinstance(self.controller, SemiLogController):
            dr_dt = self.controller.adjust_rate(self.u)

        self.r = max(0, self.r + dr_dt * self.dt)  # Ensure rate is non-negative
        self.u_history.append(self.u)
        self.r_history.append(self.r)

    def run(self, steps):
        for _ in range(steps):
            self.step()


In [24]:
def calculate_metrics(u_history, r_history, u_target, u_threshold=0.95):
    mse = np.mean((np.array(u_history) - u_target) ** 2)
    time_above_threshold = np.sum(np.array(u_history) > u_threshold)
    utilization_volatility = np.std(u_history)
    interest_rate_volatility = np.std(r_history)

    return {
        "MSE": mse,
        "Time Above Threshold": time_above_threshold,
        "Utilization Volatility": utilization_volatility,
        "Interest Rate Volatility": interest_rate_volatility,
    }


In [28]:
# Simulation parameters
params = {
    'u_init': 0.5,
    'r_init': 0.02,
    'alpha0': -3.65,
    'alpha1': 180,
    'dt': 0.01,
}

# Controllers
linear_piecewise_controller = LinearPiecewiseController(u_target=0.9, phi_params={'low': 0.9, 'high': 1.1, 'neutral': 0})
# semilog_controller = SemiLogController(u_target=0.9, phi_params={'low': 0.9, 'high': 1.1, 'neutral': 0})

# pd_controller = PDController(kp=50, kd=4, u_target=0.9)

# Simulate P Controller
sim_p = Simulation(linear_piecewise_controller, params)
for _ in range(5000):  # Run 5000 paths
    sim_p.run(steps=int(365 / params['dt']))

# # Simulate PD Controller
# sim_pd = Simulation(pd_controller, params)
# for _ in range(5000):
#     sim_pd.run(steps=int(365 / params['dt']))


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(metrics, labels):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    metric_names = ["MSE", "Time Above Threshold", "Utilization Volatility", "Interest Rate Volatility"]

    for i, ax in enumerate(axes.flatten()):
        for label, metric in metrics.items():
            ax.bar(label, metric[metric_names[i]], label=label)
        ax.set_title(metric_names[i])
        ax.legend()

    plt.tight_layout()
    plt.show()

# plot_metrics({'P Controller': metrics_p, 'PD Controller': metrics_pd}, ['P Controller', 'PD Controller'])


: 